In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [37]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import *

In [5]:
X = train.drop(['id', 'Class'], axis = 1)
y = train['Class']

In [6]:
target = test[X.columns]

In [7]:
skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)

In [8]:
cb_pred = np.zeros(target.shape[0])
for i, (tr_idx, val_idx) in enumerate(skf.split(X, y)) :
    
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    cb = CatBoostClassifier(random_state = 42, max_depth = 6, learning_rate = 0.02, iterations = 10000, use_best_model = True, eval_metric = 'Logloss')
    cb.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], early_stopping_rounds = 1000, verbose = 0)
    
    val_pred = cb.predict_proba(val_x)[:, 1]
    val_log_loss = log_loss(val_y, val_pred)
    print(f'{i + 1} Fold Logloss : {val_log_loss}')
    
    fold_pred = cb.predict_proba(target)[:, 1] / skf.n_splits
    cb_pred += fold_pred

1 Fold Logloss : 0.025108287304941544
2 Fold Logloss : 0.03374692980460994
3 Fold Logloss : 0.03013768113374496
4 Fold Logloss : 0.032806538165788934
5 Fold Logloss : 0.03146096289435348
6 Fold Logloss : 0.03067767624684073
7 Fold Logloss : 0.0329514485955561
8 Fold Logloss : 0.029677344025865058
9 Fold Logloss : 0.030984577959498123
10 Fold Logloss : 0.033879170261144696
